### 1. Import Dependencies

In [7]:
import string
import re
import nltk

nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yonti's\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2. Input data

input text

In [8]:
text = input("TEXT = ")
print(text)

hello google


import csv

In [18]:
import pandas as pd

df = pd.read_csv('data/twitter.csv')
df.head()

,text,sentiment
0,"hari ini aku masih libur, masih ada waktu semi...",1
1,yg ptm hari ini semangaattt,1
2,yg lagi ptm sini ciss dulu,1
3,yang ptm semangat ya,1
4,yang ptm selamat menikmati hari bahagia kalian ya,1


### 3. Preprocessing

In [ ]:
def preprocess(text):
    lower_text = text.lower()   # Convert text to lower

    token_text = re.sub(r"\d+", "", lower_text)    # Remove number
    token_text = re.sub('\s+',' ', token_text) # Remove multiple whitespace into single whitespace
    token_text = ''.join(c for c in token_text if c not in string.punctuation)      # Remove punctuation
    word_tokens = nltk.tokenize.word_tokenize(token_text)    # Tokenize the text
    freq_tokens = nltk.FreqDist(word_tokens)    # Frequency word token

    stop_words = [word for word in word_tokens if word not in stopwords.words('english')]   # Implement stopwords
        
    stemmer = nltk.stem.SnowballStemmer('english')  # Initialized stemmer
    stemmed_words = [stemmer.stem(word) for word in stop_words]   # Stem the words

    normalized_text = ''.join(stemmed_words)  # Join the words

"""
print('Case Folding Result : ') 
print(lower_text)

print('\nRemove punctuation, number, multiple whitespace : ')
print(token_text)

print('\nTokenizing Result : ') 
print(word_tokens)

print('\nFrequency Token : ')
print(freq_tokens.most_common())

print('\nStopword : ')
print(stop_words)

print('\nStemmer : ')
print(stemmed_words)

print('\nNormalized : ')
print(normalized_text)
"""


In [ ]:
def preprocess_data(df, lang):
    """
    Preprocesses the text in the 'review' column of the given DataFrame.
    """
    df['text'] = df['text'].apply(lambda x: preprocess(x, lang))
        
    return df

### 3. TF-IDF Vectorizer

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = preprocess_data(df, 'english')

# Create Tfidf vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Convert the 'review' column into its TF-IDF vectorized form
X = vectorizer.fit_transform(df['text'])

### 4. Train and Evaluate Data

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from textblob import TextBlob

In [10]:
# Sample text
# text = "I love this product. It's amazing!"

# Perform sentiment analysis
analysis = TextBlob(normalized_text)

# Get sentiment polarity (-1 to 1: negative to positive)
sentiment = analysis.sentiment.polarity
if sentiment > 0: 
    print("Positive sentiment")
elif sentiment < 0: 
    print("Negative sentiment")
else: 
    print("Neutral sentiment")

Neutral sentiment


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

# Convert training and test datasets into vectorized form
X_train_vectorized, vectorizer = tfidf_vectorizer(df)
X_test_vectorized, = vectorizer.transform(X_test)

# Create a Multinomial Naive Bayes model
model = MultinomialNB(alpha=1)

# Train the model
model.fit(X_train_vectorized, y_train)

# Predict sentiment for test dataset
y_pred = model.predict(X_test_vectorized)

# Calculate accuracy of the model
accuracy = accuracy_score(y_test, y_pred, normalize=True)
print(f'Accuracy of the model: {accuracy*100:.2f}%')